In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
import sys
sys.path.append("..")

In [4]:
from abstraction_validation.abstractify import abstractify_state, abstractify_boss_info
from rom_tools.rom_manager import RomManager
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms
from world_rando.coord import Coord
from abstraction_validation.sm_paths import *
from data_types.item_set import ItemSet

In [5]:
def read_state():
    ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
    return abstractify_state(ram_data, global_pos=True)

In [6]:
# #ram_data = np.memmap("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16", mode="c")#.reshape((-1, 0x2000 // 2))
# while True:
#     ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
#     #print(ram_data.shape)
#     time.sleep(0.1)
#     p = abstractify_state(ram_data, global_pos=True).position
#     #print(p)
#     map_tile = Coord(p.x / 16, p.y / 16)
#     #print(map_tile)
#     #print(position_transform(map_tile))
#     circle.location = position_transform(map_tile)

In [7]:
# Testing Lua injection
lua_fn = """
local px_addr = 0x000AF6
local py_addr = 0x000AFA
local camx_addr = 0x000911
local camy_addr = 0x000915
local camx = memory.readword(camx_addr)
local camy = memory.readword(camy_addr)
local px = memory.readword(px_addr)
local py = memory.readword(py_addr)
local screen_x = px - camx
local screen_y = py - camy
local xrad = memory.readbyte(0x7E0AFE)
local yrad = memory.readbyte(0x7E0B00)	
gui.box(screen_x + (xrad * -1), screen_y + (yrad * -1), screen_x + xrad, screen_y + yrad,"#0000FF35","#0000FFFF")
gui.text(screen_x, screen_y, "{}", 0xffffffff)
gui.text(0, 20, "{}", 0xffffffff)
"""
ss = "Hello"

In [8]:
advice_fn = """
area_offsets = {
    [0] = {[0] = 3, [1] = 10},
    [1] = {[0] = 0, [1] = 29},
    [2] = {[0] = 31, [1] = 49},
    [3] = {[0] = 37, [1] = 0},
    [4] = {[0] = 28, [1] = 29},
    [5] = {[0] = 0, [1] = 10},
    [6] = {[0] = 0, [1] = -10},
    [7] = {[0] = 0, [1] = 0}
}
local px_addr = 0x000AF6
local py_addr = 0x000AFA
local camx_addr = 0x000911
local camy_addr = 0x000915
local area_addr = 0x00079F
local mapx_addr = 0x0007A1
local mapy_addr = 0x0007A3
local camx = memory.readword(camx_addr)
local camy = memory.readword(camy_addr)
local px = memory.readword(px_addr)
local py = memory.readword(py_addr)
local screen_x = px - camx
local screen_y = py - camy
local xrad = memory.readbyte(0x7E0AFE)
local yrad = memory.readbyte(0x7E0B00)
local area = memory.readbyte(area_addr)
local mapx = memory.readbyte(mapx_addr)
local mapy = memory.readbyte(mapy_addr)
local px_global = 16 * area_offsets[area][0] + 16 * mapx + (px / 16)
local py_global = 16 * area_offsets[area][1] + 16 * mapy + (py / 16)
"""
advice2 = """
local go_x = {gox}
local go_y = {goy}
local to_x = go_x - px_global
local to_y = go_y - py_global
--print("hey")
--print(px_global, py_global)
--print(go_x, go_y)
--print(to_x, to_y)
gui.line(screen_x, screen_y, screen_x + (to_x * 16), screen_y + (to_y * 16), 0xffffffff)
local dist = math.sqrt((to_x * to_x + to_y * to_y))
gui.text(screen_x, screen_y, tostring(math.floor(dist)), 0xffffffff)
"""
def mk_advice(x, y):
    return advice_fn + advice2.format(gox=x, goy=y)

In [9]:
# while True:
#     if ss == "Hello":
#         ss = "Goodbye"
#     else:
#         ss = "Hello"
#     f = open("../../Videogame-Retrieval/snes9x_bridge.lua", "w")
#     f.write(lua_fn.format(ss, ss))
#     f.close()
#     time.sleep(1)

In [10]:
# Pre-compute node information
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")
parsed_rom = rom.parse()
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
#rooms = parse_rooms("../encoding/dsl/rooms.txt")
exits = parse_exits('../encoding/dsl/exits_for_alloy.txt')
#exits = parse_exits('../encoding/dsl/exits.txt')
all_posns = all_global_positions(rooms, parsed_rom)
world = dictify_rooms(rooms,exits)

Landing_Site 0x791f8
	 Extra: []
	 Missing: ['Landing_Site_Ship']
West_Ocean 0x793fe
	 Extra: ['West_Ocean_R4', 'West_Ocean_L']
	 Missing: ['West_Ocean_R6', 'West_Ocean_L2']
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: ['Bomb_Torizo_Bombs']
Spore_Spawn 0x79dc7
	 Extra: []
	 Missing: ['Spore_Spawn_Spore_Spawn']
Kraid 0x7a59f
	 Extra: []
	 Missing: ['Kraid_Kraid']
Statues 0x7a66a
	 Extra: []
	 Missing: ['Statues_Statues']
Crocomire 0x7a98d
	 Extra: []
	 Missing: ['Crocomire_Crocomire']
Water_Closet 0x7b1e5
	 Extra: []
	 Missing: ['Water_Closet_Drain']
Golden_Torizo 0x7b283
	 Extra: []
	 Missing: ['Golden_Torizo_Golden_Torizo']
Ridley 0x7b32e
	 Extra: []
	 Missing: ['Ridley_Ridley']
Phantoon 0x7cd13
	 Extra: []
	 Missing: ['Phantoon_Phantoon']
West_Sand_Hall 0x7d461
	 Extra: []
	 Missing: ['West_Sand_Hall_TS']
East_Sand_Hall 0x7d4c2
	 Extra: []
	 Missing: ['East_Sand_Hall_TS']
West_Sand_Hole 0x7d4ef
	 Extra: []
	 Missing: ['West_Sand_Hole_TS']
East_Sand_Hole 0x7d51e
	 Extra: 

In [11]:
# From Adam:
from collections import namedtuple

Supernode = namedtuple('Supernode', ['room', 'node', 'itemset'])

initial = Supernode('Landing_Site', 'Ship', frozenset())

def generate_links(supernode):
  room, node, itemset = supernode
  room_spec = world['Rooms'][room]
  if node in room_spec['Drops']:
    drop = room_spec['Drops'][node]
    if drop not in itemset:
      yield Supernode(room, node, itemset | frozenset([drop]))
      return # Does this mean we are forced to collect an item if available? Yes, but the graph becomes much larger...

  if node in room_spec['Doors']:
    door = room_spec['Doors'][node]
    yield Supernode(door['Room'], door['Node'], itemset)
  for e in room_spec['Edges'].get(node, []):
    terminal = e['Terminal']
    for requirements in map(frozenset,e['Requirements']):
      if requirements.issubset(itemset):
        yield Supernode(room, terminal, itemset)

nodes = set()
edges = set()

stack = []
stack.append(initial)

from tqdm.notebook import tqdm

with tqdm(total=1_023_300) as progress:
  while stack:
    u = stack.pop()
    if u not in nodes:
      progress.update(1)
      nodes.add(u)
      recent_edges = 0
      for v in generate_links(u):
        edges.add((u,v))
        recent_edges += 1
        if v not in nodes:
          stack.append(v)
      if recent_edges == 0:
        #print('dead:', u)
        pass

  0%|          | 0/1023300 [00:00<?, ?it/s]

In [12]:
def supernode_to_abstate(supernode):
    room, node, itemset = supernode
    nn = f"{room}_{node}"
    ii = ItemSet()
    for i in itemset:
        ii = ii.add(i)
    return (nn, ii)

def abstate_to_supernode(abstate):
    nn, ii = abstate
    node = nn.split("_")[-1]
    room = "_".join(nn.split("_")[:-1])
    itemset = frozenset()
    for i in ii:
        itemset |= frozenset([i])
    return Supernode(room, node, itemset)

In [13]:
len(nodes), len(edges)

(1023300, 3153239)

In [14]:
starts = [n for n in nodes if n.node == 'Ship' and not n.itemset]
finishes = [n for n in nodes if n.node == 'Ship' and 'Mother_Brain' in n.itemset]
len(starts), len(finishes)

(1, 384)

In [15]:
import networkx as nx
g = nx.DiGraph(edges)

source = "source" # a node that connects to starts
loop = "loop" # a node that allows you to restart on victory
sink = "sink" # a node that connects from finishes

for start in starts:
  g.add_edge(source, start)
  g.add_edge(loop, start)
for finish in finishes:
  g.add_edge(finish, sink)
  g.add_edge(finish, loop)

In [16]:
%%time
path = nx.shortest_path(g,source,sink)
len(path)

CPU times: user 1.02 s, sys: 0 ns, total: 1.02 s
Wall time: 1.01 s


525

In [17]:
path[19]

Supernode(room='First_Missile', node='M', itemset=frozenset({'M', 'MB'}))

In [18]:
def convert_path(path):
    new_path = []
    for p in path:
        try:
            x = supernode_to_abstate(p)
            new_path.append(x)
        except ValueError:
            pass
    return new_path

In [19]:
# Return the name of the node that Samus should visit next
def get_path_advice(state, annotated_path):
    if annotated_path is None:
        return None
    dist, node = map_state_to_node(state, all_posns)
    try:
        #print(node, state.items)
        i = annotated_path.index((node, state.items))
        if i == len(annotated_path) - 1:
            return None
        return annotated_path[i+1][0]
    # Not in list
    except ValueError:
        return None


In [20]:
all_posns["Bomb_Torizo_B"]
all_posns["Bomb_Torizo_Bombs"] = all_posns["Bomb_Torizo_B"]
del all_posns["Bomb_Torizo_B"]

In [21]:
world["Rooms"]["Bomb_Torizo"]

{'Nodes': ['L', 'Bombs'],
 'Drops': {'Bombs': 'B'},
 'Edges': {'L': [{'Terminal': 'Bombs', 'Requirements': [[]]}],
  'Bombs': [{'Terminal': 'L', 'Requirements': [[]]}]},
 'Doors': {'L': {'Room': 'BT_Flyway', 'Node': 'R', 'Direction': 'L'}}}

In [22]:
rooms["Bomb_Torizo"].mem_address

497668

In [23]:
node_memaddrs = {node : rooms[abstate_to_supernode((node, ItemSet())).room].mem_address for node in all_posns.keys()}

In [24]:
abstate_to_supernode(("Bomb_Torizo_Bombs", ItemSet()))

Supernode(room='Bomb_Torizo', node='Bombs', itemset=frozenset())

In [25]:
all_posns["Bomb_Torizo_Bombs"]

c(460,266)

In [26]:
hex(node_memaddrs["Bomb_Torizo_Bombs"])

'0x79804'

In [27]:
path[40:80]

[Supernode(room='BT_Flyway', node='L', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Parlor', node='R2', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Parlor', node='L1', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Terminator_Room', node='R', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Terminator_Room', node='L', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Green_Pirates_Shaft', node='R2', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Green_Pirates_Shaft', node='L', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Kago_Room', node='R', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Kago_Room', node='L', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Brinstar_Elevator', node='R', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Brinstar_Elevator', node='ET', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Brinstar_Entrance', node='EB', itemset=frozenset({'B', 'M', 'MB'})),
 Supernode(room='Brin

In [37]:
def abstractify_info(ram_data):
    abstate = abstractify_state(ram_data, global_pos=True)
    abboss = abstractify_boss_info(ram_data, offset=0x2000)
    abstate.items = abstate.items | abboss
    return abstate

In [29]:
def estimate_node(ram_data):
    abstate = abstractify_info(ram_data)
    current_room_a = ram_data.view("uint16")[0x79b // 2] >> 8
    current_room_b = (ram_data.view("uint16")[0x79c // 2] & 0x00ff) << 8
    current_room = current_room_a | current_room_b
    #dist, nearest_node = map_state_to_node(abstate, all_posns)
    #print(abstate.position)
    #print(dist, nearest_node)
    # Find a nearby node in the current room
    for node, pos in all_posns.items():
        node_memaddr = node_memaddrs[node]
        # Gaslighting: You're not "at" a node if we don't believe it exists...
        # Prevents things like power bomb doors from screwing with things
        if node_memaddr & 0xffff == current_room and abstate.position.euclidean(pos) < 5 and abstate_to_supernode((node, abstate.items)) in g:
            return node
    return None

In [ ]:
# X Only update position on door transition
# ? Abstraction is room-aware
# X Only update position when you're close to the node
# Abstractify can handle bosses

current_node = "source"
current_goal = "sink"
current_path = convert_path(path)
current_itemset = ItemSet()
update = False
while True:
    # Check our current node
    ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
    abstate = abstractify_info(ram_data)
    # Accurate itemset info
    if abstate.items != current_itemset:
        current_itemset = abstate.items
        print(f"Player got new item: {current_itemset}")
        update = True
    # possibly inaccurate node info
    node_estimate = estimate_node(ram_data)
    # If the node may have changed
    if node_estimate is not None:
        if current_node != node_estimate:
            print(f"Player at new node: {node_estimate}")
            current_node = node_estimate
            update = True
    if update:
        update = False
        next_node = get_path_advice(abstate, current_path)
        if next_node is not None:
            print(f"Current_goal: {next_node}")
        else:
            print("Recomputing Path...")
            # Compute a new path
            #TODO: Use estimate still to avoid e.g. spore_spawn_b
            #TODO: what happens inf player got new item but node_estimate is None?
            current_supernode = abstate_to_supernode((current_node, abstate.items))
            current_path = convert_path(nx.shortest_path(g, current_supernode, current_goal))
            next_node = get_path_advice(abstate, current_path)
            print(f"Current_goal: {next_node}")
        # Write out the new advice
        if next_node in all_posns:
            #print("YES")
            next_node_pos = all_posns[next_node]
            advice = mk_advice(next_node_pos.x, next_node_pos.y)
            f = open("../../Videogame-Retrieval/snes9x_bridge.lua", "w")
            f.write(advice)
            f.close()
    # Get the current advice
    #     f = open("../../Videogame-Retrieval/snes9x_bridge.lua", "w")
    #     f.write(lua_fn.format(ss, ss))
    #     f.close()
    time.sleep(0.1)

Player got new item: ISet(['B', 'M', 'MB', 'CB', 'E', 'Spore_Spawn'])
Player at new node: Spore_Spawn_R
Recomputing Path...
Current_goal: Spore_Spawn_Supers_L1
Player at new node: Spore_Spawn_Supers_L1
Current_goal: Spore_Spawn_Supers_S
Player at new node: Spore_Spawn_Supers_S
Current_goal: Spore_Spawn_Supers_S
Player got new item: ISet(['B', 'S', 'M', 'MB', 'CB', 'E', 'Spore_Spawn'])
Current_goal: Spore_Spawn_Supers_L2
Player at new node: Spore_Spawn_Supers_L2
Current_goal: Spore_Spawn_Farm_R
Player at new node: Spore_Spawn_Farm_R
Current_goal: Spore_Spawn_Farm_L
Player at new node: Spore_Spawn_Farm_L
Current_goal: Big_Pink_R3
Player at new node: Big_Pink_R3
Current_goal: Big_Pink_R4
Player at new node: Big_Pink_M2
Recomputing Path...
Current_goal: Big_Pink_R4
Player at new node: Big_Pink_R4
Current_goal: Green_Hill_Zone_L
Player at new node: Green_Hill_Zone_L
Current_goal: Green_Hill_Zone_R2
Player at new node: Green_Hill_Zone_R2
Current_goal: Noob_Bridge_L
Player at new node: Noob_B

In [158]:
%%time
nx.shortest_path(g, current_supernode, current_goal)

KeyboardInterrupt: 

In [157]:
%%time
nx.shortest_path(g,source,sink)


KeyboardInterrupt



In [53]:
ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
a = abstractify_info(ram_data)

In [55]:
a

(c(1111,568)|V: 0, H: RUN, 0|STAND|ISet(['B', 'PB', 'S', 'M', 'V', 'GS', 'SB', 'HJ', 'MB', 'CB', 'WB', 'E', 'Spazer', 'IB', 'Kraid', 'Phantoon', 'Botwoon', 'Spore_Spawn', 'Ceres_Ridley']))